# Gridsearch Cluster all small dag generation evaluation

In [1]:
%load_ext autoreload
%autoreload 2
from clustercausal.experiments.ExperimentRunner import ExperimentRunner
import itertools
import yaml
import os
import pickle
import pandas as pd
from clustercausal.utils.Utils import *
from clustercausal.experiments.Utils import load_experiment_folder
pd.set_option('display.max_columns', None)

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
2023-10-06 11:22:08,824 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\rpy2\situation.py[line:41] - INFO: cffi mode is CFFI_MODE.ANY
2023-10-06 11:22:08,831 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\rpy2\situation.py[line:218] - INFO: R home found: C:\Program Files\R\R-4.3.1
2023-10-06 11:22:08,852 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\rpy2\rinterface_lib\embedded.py[line:20] - INFO: Default options to initialize R: rpy2, --quiet, --no-save
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# Can take a longer time depending on the numer of experiments, ca. 5sec for 100 experiments
data = load_experiment_folder('clustercausal\experiments\_results\Cluster_PC_small_all_dag_generation')

### Data exploration

In [3]:
data[:5]

,base_adj_f1_score,base_adj_false_negative,base_adj_false_positive,base_adj_precision,base_adj_recall,base_adj_true_negative,base_adj_true_positive,base_arrow_f1_score,base_arrow_f1_score_ce,base_arrow_false_negative,base_arrow_false_negative_ce,base_arrow_false_positive,base_arrow_false_positive_ce,base_arrow_precision,base_arrow_precision_ce,base_arrow_recall,base_arrow_recall_ce,base_arrow_true_negative,base_arrow_true_negative_ce,base_arrow_true_positive,base_arrow_true_positive_ce,base_base_shd,base_sid_lower,cluster_adj_f1_score,cluster_adj_false_negative,cluster_adj_false_positive,cluster_adj_precision,cluster_adj_recall,cluster_adj_true_negative,cluster_adj_true_positive,cluster_arrow_f1_score,cluster_arrow_f1_score_ce,cluster_arrow_false_negative,cluster_arrow_false_negative_ce,cluster_arrow_false_positive,cluster_arrow_false_positive_ce,cluster_arrow_precision,cluster_arrow_precision_ce,cluster_arrow_recall,cluster_arrow_recall_ce,cluster_arrow_true_negative,cluster_arrow_true_negative_ce,cluster_arrow_true_positive,cluster_arrow_true_positive_ce,cluster_cluster_shd,cluster_sid_lower,pruned_base_adj_f1_score,pruned_base_adj_false_negative,pruned_base_adj_false_positive,pruned_base_adj_precision,pruned_base_adj_recall,pruned_base_adj_true_negative,pruned_base_adj_true_positive,pruned_base_arrow_f1_score,pruned_base_arrow_f1_score_ce,pruned_base_arrow_false_negative,pruned_base_arrow_false_negative_ce,pruned_base_arrow_false_positive,pruned_base_arrow_false_positive_ce,pruned_base_arrow_precision,pruned_base_arrow_precision_ce,pruned_base_arrow_recall,pruned_base_arrow_recall_ce,pruned_base_arrow_true_negative,pruned_base_arrow_true_negative_ce,pruned_base_arrow_true_positive,pruned_base_arrow_true_positive_ce,pruned_base_pruned_base_shd,pruned_base_sid_lower,alpha,dag_method,distribution_type,edge_ratios,empty_graph_shd,indep_test,n_clusters,n_edges,n_nodes,noise_scale,sample_size,scm_method,seed,true_sid_lower,true_sid_upper,weight_range
0,1.000000,0.0,0.0,1.000000,1.000000,41.0,4.0,NaN,NaN,4.0,4.0,0.0,0.0,NaN,NaN,0.000000,0.000000,96.0,96.0,0.0,0.0,0.0,NaN,1.000000,0.0,0.0,1.000000,1.000000,41.0,4.0,1.000000,1.000000,0.0,0.0,0.0,0.0,1.0000,1.000000,1.00,1.000000,96.0,96.0,4.0,4.0,0.0,NaN,1.000000,0.0,0.0,1.000000,1.000000,41.0,4.0,NaN,NaN,4.0,4.0,0.0,0.0,NaN,NaN,0.000000,0.000000,96.0,96.0,0.0,0.0,0.0,NaN,0.01,erdos_renyi,exp,"[0.05, 0.12, 0.12]",4.0,fisherz,2.0,10.0,10.0,1.0,1000.0,linear,None,NaN,NaN,"(-1, 2)"
1,0.857143,3.0,0.0,1.000000,0.750000,33.0,9.0,0.736842,0.875000,5.0,2.0,0.0,0.0,1.000000,1.000000,0.583333,0.777778,88.0,91.0,7.0,7.0,3.0,NaN,0.857143,3.0,0.0,1.000000,0.750000,33.0,9.0,0.857143,1.000000,3.0,0.0,0.0,0.0,1.0000,1.000000,0.75,1.000000,88.0,91.0,9.0,9.0,3.0,NaN,0.857143,3.0,0.0,1.000000,0.750000,33.0,9.0,0.736842,0.875000,5.0,2.0,0.0,0.0,1.000000,1.000000,0.583333,0.777778,88.0,91.0,7.0,7.0,3.0,NaN,0.05,erdos_renyi,exp,"[0.14, 0.29, 0.29]",12.0,fisherz,2.0,10.0,10.0,1.0,1000.0,linear,None,NaN,NaN,"(-1, 2)"
2,0.857143,0.0,2.0,0.750000,1.000000,37.0,6.0,0.181818,0.222222,5.0,5.0,4.0,2.0,0.200000,0.333333,0.166667,0.166667,90.0,92.0,1.0,1.0,4.0,NaN,0.923077,0.0,1.0,0.857143,1.000000,38.0,6.0,0.500000,0.545455,3.0,3.0,3.0,2.0,0.5000,0.600000,0.50,0.500000,91.0,92.0,3.0,3.0,3.0,NaN,0.857143,0.0,2.0,0.750000,1.000000,37.0,6.0,0.181818,0.222222,5.0,5.0,4.0,2.0,0.200000,0.333333,0.166667,0.166667,90.0,92.0,1.0,1.0,4.0,NaN,0.10,erdos_renyi,exp,"[0.31, 0.22, 0.22]",6.0,fisherz,2.0,10.0,10.0,1.0,1000.0,linear,None,NaN,NaN,"(-1, 2)"
3,0.888889,0.0,2.0,0.800000,1.000000,35.0,8.0,0.500000,0.571429,4.0,4.0,4.0,2.0,0.500000,0.666667,0.500000,0.500000,88.0,90.0,4.0,4.0,4.0,NaN,0.888889,0.0,2.0,0.800000,1.000000,35.0,8.0,0.666667,0.750000,2.0,2.0,4.0,2.0,0.6000,0.750000,0.75,0.750000,88.0,90.0,6.0,6.0,4.0,NaN,0.888889,0.0,2.0,0.800000,1.000000,35.0,8.0,0.500000,0.571429,4.0,4.0,4.0,2.0,0.500000,0.666667,0.500000,0.500000,88.0,90.0,4.0,4.0,4.0,NaN,0.25,erdos_renyi,exp,"[0.09, 0.19, 0.19]",8.0,fisherz,2.0,10.0,10.

In [4]:
data.describe()

,base_adj_f1_score,base_adj_false_negative,base_adj_false_positive,base_adj_precision,base_adj_recall,base_adj_true_negative,base_adj_true_positive,base_arrow_f1_score,base_arrow_f1_score_ce,base_arrow_false_negative,base_arrow_false_negative_ce,base_arrow_false_positive,base_arrow_false_positive_ce,base_arrow_precision,base_arrow_precision_ce,base_arrow_recall,base_arrow_recall_ce,base_arrow_true_negative,base_arrow_true_negative_ce,base_arrow_true_positive,base_arrow_true_positive_ce,base_base_shd,base_sid_lower,cluster_adj_f1_score,cluster_adj_false_negative,cluster_adj_false_positive,cluster_adj_precision,cluster_adj_recall,cluster_adj_true_negative,cluster_adj_true_positive,cluster_arrow_f1_score,cluster_arrow_f1_score_ce,cluster_arrow_false_negative,cluster_arrow_false_negative_ce,cluster_arrow_false_positive,cluster_arrow_false_positive_ce,cluster_arrow_precision,cluster_arrow_precision_ce,cluster_arrow_recall,cluster_arrow_recall_ce,cluster_arrow_true_negative,cluster_arrow_true_negative_ce,cluster_arrow_true_positive,cluster_arrow_true_positive_ce,cluster_cluster_shd,cluster_sid_lower,pruned_base_adj_f1_score,pruned_base_adj_false_negative,pruned_base_adj_false_positive,pruned_base_adj_precision,pruned_base_adj_recall,pruned_base_adj_true_negative,pruned_base_adj_true_positive,pruned_base_arrow_f1_score,pruned_base_arrow_f1_score_ce,pruned_base_arrow_false_negative,pruned_base_arrow_false_negative_ce,pruned_base_arrow_false_positive,pruned_base_arrow_false_positive_ce,pruned_base_arrow_precision,pruned_base_arrow_precision_ce,pruned_base_arrow_recall,pruned_base_arrow_recall_ce,pruned_base_arrow_true_negative,pruned_base_arrow_true_negative_ce,pruned_base_arrow_true_positive,pruned_base_arrow_true_positive_ce,pruned_base_pruned_base_shd,pruned_base_sid_lower,alpha,empty_graph_shd,n_clusters,n_edges,n_nodes,noise_scale,sample_size,true_sid_lower,true_sid_upper
count,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,533.000000,533.000000,540.000000,540.000000,540.000000,540.000000,534.000000,534.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,0.0,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,0.0,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,533.000000,533.000000,540.000000,540.000000,540.000000,540.000000,534.000000,534.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,0.0,540.00000,540.000000,540.000000,540.000000,540.0,540.0,540.0,0.0,0.0
mean,0.684922,11.627778,1.496296,0.892717,0.594730,20.085185,11.790741,0.435537,0.644833,16.357407,4.729630,3.970370,2.681481,0.656093,0.732158,0.344404,0.582741,72.611111,85.527778,7.061111,7.061111,15.805556,NaN,0.725079,9.894444,1.966667,0.875195,0.655716,19.614815,13.524074,0.649790,0.893540,11.605556,1.711111,2.696296,0.896296,0.815542,0.926955,0.571827,0.867484,73.885185,85.579630,11.812963,11.812963,12.757407,NaN,0.684922,11.627778,1.496296,0.892717,0.594730,20.085185,11.790741,0.435537,0.644833,16.357407,4.729630,3.970370,2.681481,0.656093,0.732158,0.344404,0.582741,72.611111,85.527778,7.061111,7.061111,15.805556,NaN,0.18200,23.418519,3.500000,20.000000,10.0,1.0,1000.0,NaN,NaN
std,0.153518,9.171848,1.623684,0.109553,0.218143,10.839706,3.017375,0.175547,0.189815,9.650480,2.278611,2.926282,2.205245,0.217215,0.206494,0.180314,0.205566,11.713837,4.346022,3.386370,3.386370,9.785509,NaN,0.128116,8.022115,1.951832,0.119320,0.196531,10.706047,4.261552,0.157526,0.110947,8.460427,1.592499,2.312422,1.179755,0.149801,0.098354,0.200928,0.130227,11.061548,4.535896,4.397695,4.397695,8.156769,NaN,0.153518,9.171848,1.623684,0.109553,0.218143,10.839706,3.017375,0.175547,0.189815,9.650480,2.278611,2.926282,2.205245,0.217215,0.206494,0.180314,0.205566,11.713837,4.346022,3.38637

In [5]:
prec_recall_metrics = ['base_adj_precision', 'pruned_base_adj_precision', 'cluster_adj_precision', \
           'base_adj_recall', 'pruned_base_adj_recall','cluster_adj_recall', \
           'base_adj_f1_score', 'pruned_base_adj_f1_score', 'cluster_adj_f1_score']
data[prec_recall_metrics].describe()

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
count,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000
mean,0.892717,0.892717,0.875195,0.594730,0.594730,0.655716,0.684922,0.684922,0.725079
std,0.109553,0.109553,0.119320,0.218143,0.218143,0.196531,0.153518,0.153518,0.128116
min,0.363636,0.363636,0.384615,0.230769,0.230769,0.256410,0.360000,0.360000,0.400000
25%,0.846154,0.846154,0.812500,0.405405,0.405405,0.496795,0.561053,0.561053,0.631092
50%,0.916667,0.916667,0.900000,0.555556,0.555556,0.627315,0.666667,0.666667,0.714286
75%,1.000000,1.000000,0.956522,0.761905,0.761905,0.809524,0.814815,0.814815,0.823529
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
count_metrics = ['base_adj_false_negative', 'pruned_base_adj_false_negative', 'cluster_adj_false_negative', \
                 'base_adj_false_positive', 'pruned_base_adj_false_positive','cluster_adj_false_positive', \
                    'base_adj_true_negative', 'pruned_base_adj_true_negative', 'cluster_adj_true_negative', \
                        'base_adj_true_positive', 'pruned_base_adj_true_positive', 'cluster_adj_true_positive']
data[count_metrics]

,base_adj_false_negative,pruned_base_adj_false_negative,cluster_adj_false_negative,base_adj_false_positive,pruned_base_adj_false_positive,cluster_adj_false_positive,base_adj_true_negative,pruned_base_adj_true_negative,cluster_adj_true_negative,base_adj_true_positive,pruned_base_adj_true_positive,cluster_adj_true_positive
0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,41.0,41.0,4.0,4.0,4.0
1,3.0,3.0,3.0,0.0,0.0,0.0,33.0,33.0,33.0,9.0,9.0,9.0
2,0.0,0.0,0.0,2.0,2.0,1.0,37.0,37.0,38.0,6.0,6.0,6.0
3,0.0,0.0,0.0,2.0,2.0,2.0,35.0,35.0,35.0,8.0,8.0,8.0
4,3.0,3.0,3.0,4.0,4.0,5.0,27.0,27.0,26.0,11.0,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
535,9.0,9.0,7.0,0.0,0.0,1.0,24.0,24.0,23.0,12.0,12.0,14.0
536,11.0,11.0,9.0,1.0,1.0,5.0,23.0,23.0,19.0,10.0,10.0,12.0
537,9.0,9.0,5.0,2.0,2.0,4.0,22.0,22.0,20.0,12.0,12.0,16.0
538,4.0,4.0,1.0,0.0,0.0,0.0,24.0,24.0,24.0,17.0,17.0,20.0


In [7]:
dist_metrics = ['base_base_shd', 'pruned_base_pruned_base_shd', 'cluster_cluster_shd']#, \
         #   'base_sid_lower', 'base_sid_upper', 'pruned_base_sid_lower', 'pruned_base_sid_upper', \
         #      'cluster_sid_lower', 'cluster_sid_upper']
data[dist_metrics].describe()

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
count,540.000000,540.000000,540.000000
mean,15.805556,15.805556,12.757407
std,9.785509,9.785509,8.156769
min,0.000000,0.000000,0.000000
25%,7.750000,7.750000,6.000000
50%,15.000000,15.000000,12.000000
75%,25.000000,25.000000,20.000000
max,36.000000,36.000000,34.000000


In [8]:
edge_ratios = data['edge_ratios'].apply(pd.Series)
edge_ratios.columns = ['intra_cluster', 'inter_cluster', 'inter_including_disconnected_cluster']
mean_values = edge_ratios.mean()
mean_values = pd.DataFrame([mean_values])
print('Edge Ratios \n', mean_values)

Edge Ratios 
    intra_cluster  inter_cluster  inter_including_disconnected_cluster
0       0.438167       0.603444                              0.543148


### Comparing true & false positives and true & false negatives

In [9]:
true_false_positive_metrics = ['base_adj_true_positive', 'pruned_base_adj_true_positive', 'cluster_adj_true_positive', \
                          'base_adj_false_positive', 'pruned_base_adj_false_positive','cluster_adj_false_positive']

true_false_negative_metrics = ['base_adj_true_negative', 'pruned_base_adj_true_negative', 'cluster_adj_true_negative', \
                          'base_adj_false_negative', 'pruned_base_adj_false_negative', 'cluster_adj_false_negative']

true_false_metrics = ['base_adj_true_positive', 'pruned_base_adj_true_positive', 'cluster_adj_true_positive', \
                    'base_adj_true_negative', 'pruned_base_adj_true_negative', 'cluster_adj_true_negative', \
                    'base_adj_false_positive', 'pruned_base_adj_false_positive','cluster_adj_false_positive', \
                    'base_adj_false_negative', 'pruned_base_adj_false_negative', 'cluster_adj_false_negative']

In [10]:
data[true_false_positive_metrics].describe()

,base_adj_true_positive,pruned_base_adj_true_positive,cluster_adj_true_positive,base_adj_false_positive,pruned_base_adj_false_positive,cluster_adj_false_positive
count,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000
mean,11.790741,11.790741,13.524074,1.496296,1.496296,1.966667
std,3.017375,3.017375,4.261552,1.623684,1.623684,1.951832
min,4.000000,4.000000,4.000000,0.000000,0.000000,0.000000
25%,10.000000,10.000000,11.000000,0.000000,0.000000,1.000000
50%,12.000000,12.000000,13.000000,1.000000,1.000000,1.000000
75%,14.000000,14.000000,16.000000,2.000000,2.000000,3.000000
max,21.000000,21.000000,27.000000,12.000000,12.000000,12.000000


In [11]:
data[true_false_negative_metrics].describe()

,base_adj_true_negative,pruned_base_adj_true_negative,cluster_adj_true_negative,base_adj_false_negative,pruned_base_adj_false_negative,cluster_adj_false_negative
count,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000
mean,20.085185,20.085185,19.614815,11.627778,11.627778,9.894444
std,10.839706,10.839706,10.706047,9.171848,9.171848,8.022115
min,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000
25%,8.000000,8.000000,8.000000,3.000000,3.000000,3.000000
50%,22.000000,22.000000,21.000000,10.000000,10.000000,8.000000
75%,29.000000,29.000000,28.000000,21.000000,21.000000,17.000000
max,41.000000,41.000000,41.000000,30.000000,30.000000,29.000000


### Comparing total true and false

In [12]:
data['base_total_true'] = data['base_adj_true_positive'] + data['base_adj_true_negative']
data['base_total_false'] = data['base_adj_false_positive'] + data['base_adj_false_negative']
data['pruned_base_total_true'] = data['pruned_base_adj_true_positive'] + data['pruned_base_adj_true_negative']
data['pruned_base_total_false'] = data['pruned_base_adj_false_positive'] + data['pruned_base_adj_false_negative']
data['cluster_total_true'] = data['cluster_adj_true_positive'] + data['cluster_adj_true_negative']
data['cluster_total_false'] = data['cluster_adj_false_positive'] + data['cluster_adj_false_negative']
total_true_false_metrics = ['base_total_true', 'pruned_base_total_true', 'cluster_total_true', \
                    'base_total_false', 'pruned_base_total_false', 'cluster_total_false']
data[total_true_false_metrics].describe()

,base_total_true,pruned_base_total_true,cluster_total_true,base_total_false,pruned_base_total_false,cluster_total_false
count,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000
mean,31.875926,31.875926,33.138889,13.124074,13.124074,11.861111
std,9.055660,9.055660,7.809250,9.055660,9.055660,7.809250
min,13.000000,13.000000,15.000000,0.000000,0.000000,0.000000
25%,22.000000,22.000000,26.000000,5.000000,5.000000,5.000000
50%,34.000000,34.000000,34.000000,11.000000,11.000000,11.000000
75%,40.000000,40.000000,40.000000,23.000000,23.000000,19.000000
max,45.000000,45.000000,45.000000,32.000000,32.000000,30.000000


### Grouping by clusters

In [13]:
data.groupby('n_clusters').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
n_clusters,,,,,,,,,
2.0,0.884788,0.884788,0.875986,0.595990,0.595990,0.615012,0.681392,0.681392,0.693473
3.0,0.886431,0.886431,0.868753,0.590789,0.590789,0.636543,0.680379,0.680379,0.708696
4.0,0.896071,0.896071,0.878933,0.598194,0.598194,0.672427,0.687441,0.687441,0.738802
5.0,0.903578,0.903578,0.877108,0.593945,0.593945,0.698882,0.690475,0.690475,0.759346


In [14]:
data.groupby('n_clusters').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
n_clusters,,,
2.0,15.940741,15.940741,14.548148
3.0,16.074074,16.074074,13.533333
4.0,15.622222,15.622222,12.066667
5.0,15.585185,15.585185,10.881481


### Grouping by alpha

In [15]:
data.groupby('alpha').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
alpha,,,,,,,,,
0.01,0.945834,0.945834,0.943129,0.536740,0.536740,0.604922,0.663215,0.663215,0.720667
0.05,0.923129,0.923129,0.900883,0.563566,0.563566,0.627362,0.677199,0.677199,0.722027
0.10,0.914484,0.914484,0.897258,0.576192,0.576192,0.631867,0.683461,0.683461,0.723626
0.25,0.879543,0.879543,0.857882,0.631334,0.631334,0.689349,0.707339,0.707339,0.739917
0.50,0.800595,0.800595,0.776823,0.665817,0.665817,0.725081,0.693394,0.693394,0.719158


In [16]:
data.groupby('alpha').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
alpha,,,
0.01,15.648148,15.648148,12.574074
0.05,15.814815,15.814815,13.046296
0.10,15.555556,15.555556,12.509259
0.25,15.592593,15.592593,12.203704
0.50,16.416667,16.416667,13.453704


### Grouping by distribution type


In [17]:
data.groupby('distribution_type').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
distribution_type,,,,,,,,,
exp,0.896182,0.896182,0.877982,0.590630,0.590630,0.651269,0.682330,0.682330,0.723760
gauss,0.885649,0.885649,0.870449,0.597399,0.597399,0.659418,0.684039,0.684039,0.724638
gumbel,0.896320,0.896320,0.877154,0.596159,0.596159,0.656461,0.688396,0.688396,0.726840


In [18]:
data.groupby('distribution_type').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
distribution_type,,,
exp,15.977778,15.977778,12.888889
gauss,15.722222,15.722222,12.694444
gumbel,15.716667,15.716667,12.688889


### Grouping by DAG method

In [19]:
data.groupby('dag_method').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
dag_method,,,,,,,,,
erdos_renyi,0.874319,0.874319,0.847401,0.678213,0.678213,0.730074,0.741834,0.741834,0.765118
hierarchical,0.924899,0.924899,0.931988,0.377884,0.377884,0.471990,0.533151,0.533151,0.621541
scale_free,0.878933,0.878933,0.846196,0.728092,0.728092,0.765085,0.779780,0.779780,0.788578


In [20]:
data.groupby('dag_method').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
dag_method,,,
erdos_renyi,11.916667,11.916667,9.450000
hierarchical,26.733333,26.733333,21.688889
scale_free,8.766667,8.766667,7.133333


### Grouping by #clusters

In [21]:
data.groupby('n_clusters').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
n_clusters,,,,,,,,,
2.0,0.884788,0.884788,0.875986,0.595990,0.595990,0.615012,0.681392,0.681392,0.693473
3.0,0.886431,0.886431,0.868753,0.590789,0.590789,0.636543,0.680379,0.680379,0.708696
4.0,0.896071,0.896071,0.878933,0.598194,0.598194,0.672427,0.687441,0.687441,0.738802
5.0,0.903578,0.903578,0.877108,0.593945,0.593945,0.698882,0.690475,0.690475,0.759346


In [22]:
data.groupby('n_clusters').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
n_clusters,,,
2.0,15.940741,15.940741,14.548148
3.0,16.074074,16.074074,13.533333
4.0,15.622222,15.622222,12.066667
5.0,15.585185,15.585185,10.881481


### Grouping by #edges

In [23]:
data.groupby('n_edges').mean()[prec_recall_metrics]

,base_adj_precision,pruned_base_adj_precision,cluster_adj_precision,base_adj_recall,pruned_base_adj_recall,cluster_adj_recall,base_adj_f1_score,pruned_base_adj_f1_score,cluster_adj_f1_score
n_edges,,,,,,,,,
10.0,0.867037,0.867037,0.852442,0.716195,0.716195,0.762076,0.742530,0.742530,0.769993
20.0,0.905711,0.905711,0.881158,0.582347,0.582347,0.649551,0.690230,0.690230,0.730515
30.0,0.905403,0.905403,0.891985,0.485647,0.485647,0.555521,0.622006,0.622006,0.674729


In [24]:
data.groupby('n_edges').mean()[dist_metrics]

,base_base_shd,pruned_base_pruned_base_shd,cluster_cluster_shd
n_edges,,,
10.0,11.783333,11.783333,9.611111
20.0,15.216667,15.216667,12.333333
30.0,20.416667,20.416667,16.327778
